In [ ]:
import time, threading, queue, sys, gc
from typing import Tuple

import numpy as np
import pyqtgraph as pg
from PyQt6 import QtCore
from collections import deque

import logging, logging.handlers

import h5py, pathlib
from datetime import datetime

| **速度目標 [rps]** | **1秒当たり [pulses / s]** | **1iterあたり [pulses]** | **カウント** | **再構成速度 (UP / DOWN) \[rps]** | **偏差 \[%]** |
| ----------------- | -------------------------- | --------------------------- | ----------------------- | ------------------------------- | -------------------------- |
| 0.1               | 204.8                      | 25.6                        | 26<br>25                | 0.102<br>0.09765625             | +1.56<br>−2.34             |
| 0.2               | 409.6                      | 51.2                        | 52<br>51                | 0.203<br>0.19921875             | +1.56<br>−0.39             |
| 0.3               | 614.4                      | 76.8                        | 77<br>76                | 0.301<br>0.296875               | +0.26<br>−1.04             |
| 0.4               | 819.2                      | 102.4                       | 103<br>102              | 0.402<br>0.3984375              | +0.59<br>−0.39             |
| 0.5               | 1 024.0                    | 128.0                       | 128<br>128              | 0.500<br>0.500                  | 0.00<br>0.00               |
| 0.6               | 1 228.8                    | 153.6                       | 154<br>153              | 0.602<br>0.59765625             | +0.26<br>−0.39             |
| 0.7               | 1 433.6                    | 179.2                       | 180<br>179              | 0.703<br>0.69921875             | +0.45<br>−0.11             |
| 0.8               | 1 638.4                    | 204.8                       | 205<br>204              | 0.801<br>0.796875               | +0.10<br>−0.39             |
| 0.9               | 1 843.2                    | 230.4                       | 231<br>230              | 0.902<br>0.8984375              | +0.26<br>−0.17             |
| 1.0               | 2 048.0                    | 256.0                       | 256<br>256              | 1.000<br>1.000                  | 0.00<br>0.00               |


In [2]:
# ------------------------------------------------------------  parameters
DEBUG = True  # set True for jitter log
SAMPLE_RATE = 100_000  # Hz
GEN_CHUNK_SEC = 0.1 # corresponds to PROC_INTERVAL up to 0.05
PROC_INTERVAL = 0.125          # process rate

CHUNK_SEC     = GEN_CHUNK_SEC
N_SAMPLES_GEN = int(SAMPLE_RATE * GEN_CHUNK_SEC) # 100_000 * 0.05 = 5000
SAMPLES_PROC = int(PROC_INTERVAL * SAMPLE_RATE) # 0.125s * 100kHz = 12500
QUEUE_DEPTH = 40  # raw AB backlog
QUAD_DEPTH = 40  # processed backlog (same)
RUN_SEC = 600  # auto‑stop after 10 s
DISPLAY_SEC = RUN_SEC+600
PLOT_SEC = 0.02  # *** fixed x-axis window width (s) ***
GUI_INTERVAL_MS = 50  # update interval (ms) e.g. 50ms = 20Hz

PULSE_HEIGHT = 5.0  # amplitude
INPUT_VELOCITY = 0.1  # rps

PULSE_WIDTH = 1 / (INPUT_VELOCITY * 512)  # period  (s)
PULSE_DUTY = 0.5  # duty
PULSE_PHASE_A = 0.0  # phase offset (s)
PULSE_PHASE_B = -PULSE_WIDTH / 4  # phase offset (s)

QUADPULSE_WIDTH = 0.00025  # width (s) assuming 4x given 1 rps
THRESHOLD_DEFAULT = 2.5  # logic threshold (V)

IDEAL_CPS = INPUT_VELOCITY   # counts/s (constant ideal)

# GUI buffer lengths (empirical)
HISTORY = int(SAMPLE_RATE * PLOT_SEC)
COUNT_HISTORY = int(RUN_SEC / GEN_CHUNK_SEC)
VELO_HISTORY = COUNT_HISTORY

# ------------------------------------------------------------  logging
LOG_CHUNK = 1024       # flush log queue every LOG_CHUNK messages




# ------------------------------------------------------------  queues & stop flag
buf_q = queue.Queue(maxsize=QUEUE_DEPTH)  # raw (t, A, B)
quad_q = queue.Queue(maxsize=QUAD_DEPTH)  # processed (t, A, B, quad)
stop_writer = threading.Event()

log_q = queue.Queue(maxsize=0)
queue_h = logging.handlers.QueueHandler(log_q)
logger = logging.getLogger("debug")
logger.setLevel(logging.INFO)
logger.addHandler(queue_h)

In [3]:
run_dir = pathlib.Path("../runs")
run_dir.mkdir(exist_ok=True)
timestamp = datetime.now().strftime("%y%m%d%H%M%S")
h5path = f"{run_dir}/{timestamp}.h5"

h5f = h5py.File(h5path, "w")
dset = h5f.create_dataset(
    "log", shape=(0, 3), maxshape=(None, 3),
    dtype=np.float32, chunks=(LOG_CHUNK, 3), compression="gzip"
)

print("dset shape  =", dset.shape)
print("dset maxshape =", dset.maxshape)

dset shape  = (0, 3)
dset maxshape = (None, 3)


In [4]:
def gen_chunk_pulse(
    t: np.ndarray,
    *,
    height: float = PULSE_HEIGHT,
    width: float = PULSE_WIDTH,
    duty: float = PULSE_DUTY,
    phase: float = 0.0,
) -> np.ndarray:
    mod = (t + phase) % width
    return np.where(mod < duty * width, height, 0.0).astype(np.float32)


REL_AXIS_GEN = (
    np.arange(N_SAMPLES_GEN, dtype=np.float32) / SAMPLE_RATE
)  # 0 ... 0.2 s

# ------------------------------------------------------------  AB → direction → quad helpers


def gen_pulse_direction(
    dA: np.ndarray,
    dB: np.ndarray,
    *,
    threshold: float,
    prev_A: bool | None = None,
    prev_B: bool | None = None,
) -> tuple[np.ndarray, bool, bool]:
    """
    Detect direction for one block **with perfect edge coverage**.

    Parameters
    ----------
    dA, dB : ndarray[float32]
        Analog levels of phase-A / phase-B for the current block.
    threshold : float
        Logic threshold [V].
    prev_A, prev_B : bool | None
        Logical state of A/B *at the end of the PREVIOUS block*.
        • If None (first block), the function falls back to the
          “self-shift” method used before.

    Returns
    -------
    dir_log : ndarray[int8]
        +1 = CW edge, –1 = CCW edge, 0 = no edge.
    last_A, last_B : bool
        Logical state of A/B at the *end* of this block — feed these
        into the next call to avoid losing the boundary edge.
    """
    # --- current logic level ----------------------------------------
    A = dA > threshold
    B = dB > threshold

    # --- previous sample for XOR -----------------------------------
    if prev_A is None:  # first block → old behaviour
        A_prev = np.concatenate(([A[0]], A[:-1]))
        B_prev = np.concatenate(([B[0]], B[:-1]))
    else:               # use states carried over from last block
        A_prev = np.concatenate(([prev_A], A[:-1]))
        B_prev = np.concatenate(([prev_B], B[:-1]))

    dir_log = (B_prev ^ A).astype(int) - (A_prev ^ B).astype(int)

    return dir_log.astype(np.int8), bool(A[-1]), bool(B[-1])

def pulse_count(dir_log: np.ndarray) -> int:
    #return int(np.sum(dir_log))
    return np.sum(dir_log)


def gen_quad_pulse(
    t: np.ndarray, dir_log: np.ndarray, width: float, height: float, sampling_rate: int
) -> np.ndarray:
    samples = int(width * sampling_rate)
    if samples <= 0:
        return np.zeros_like(dir_log, dtype=np.float32)
    base = np.full(samples, height, dtype=np.float32)
    return np.convolve(dir_log, base, mode="full")[: len(t)]


# ------------------------------------------------------------  producer thread
def generator() -> None:
    """Generate AB rectangular-wave chunks at real-time cadence."""
    chunk_idx = 0
    next_t = time.perf_counter()
    while not stop_writer.is_set():
        base = chunk_idx * GEN_CHUNK_SEC
        t_axis = REL_AXIS_GEN + base
        pulse_A = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_A)
        pulse_B = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_B)
        try:
            buf_q.put_nowait((t_axis, pulse_A, pulse_B))
        except queue.Full:
            pass

        chunk_idx += 1
        next_t += GEN_CHUNK_SEC
        sleep = next_t - time.perf_counter()
        if sleep > 0:
            time.sleep(sleep)
        else:
            next_t = time.perf_counter()


# ------------------------------------------------------------  consumer thread
def log_listener():
    handler = logging.StreamHandler(sys.stdout)
    listener = logging.handlers.QueueListener(log_q, handler)
    listener.start()
    stop_writer.wait()
    listener.stop()

def command_freq(t_DC, step, rst, target_freq, proc_interval):

    time = np.arange(0, t_DC+36000*target_freq/(step/rst)+60*60*2, proc_interval, dtype=np.float32)
    freq = np.zeros(int((t_DC+36000*target_freq/(step/rst)+60*60*2)*(1/proc_interval)), dtype=np.float32)

    # Refactoring suggestions
    # 1. Use numpy's vectorized operations instead of loops for better performance.
    # eg. precomupte the indices and use numpy's advanced indexing
    # n_dc       = int(np.round(t_dc / sample_dt))
    # freq[:n_dc] = 0.0

    for i in range(int(t_DC*(1/proc_interval))): #DCの速度指令値=0rps
        freq[i] = 0.0

    for i in range(int(36000*target_freq/step)): #加速中の速度指令値
        for j in range(int(rst*(1/proc_interval))):
            freq[int((t_DC+i*rst)*(1/proc_interval)+j)] = target_freq/36000*step*(i+1)

    for i in range(int(60*60*2*(1/proc_interval))): #Stableの速度指令値
        freq[int((t_DC+36000*target_freq/(step/rst))*(1/proc_interval))+i] = target_freq

    return time, freq

class IdealVelocityProvider:
    def __init__(self, duration: float, dt: float) -> None:
        #num_points: int = int(duration / dt) + 1
        #self.time_axis: np.ndarray = np.linspace(0.0, duration, num_points, dtype=np.float32)
        #self.velocity:  np.ndarray = np.full(num_points, IDEAL_CPS, dtype=np.float32)
        self.time_axis, self.velocity = command_freq(90, 5, 1, 1, proc_interval=PROC_INTERVAL)

    def slice(self, t_start: float, t_end: float) -> tuple[np.ndarray, np.ndarray]:
        """Return the time and velocity arrays that fall within [t_start, t_end]."""
        idx_start, idx_end = np.searchsorted(self.time_axis, [t_start, t_end])
        return self.time_axis[idx_start:idx_end], self.velocity[idx_start:idx_end]

ideal: IdealVelocityProvider | None = None  # initialised lazily in processor()


def processor() -> None:
    """
    deque ベース（旧方式）に戻したリアルタイム・コンシューマ。
    - buf_q から取り出した A/B チャンクを deque(Ring) に追加
    - PROC_INTERVAL ごとに SAMPLES_PROC 分を popleft
    - 理想速度 (IdealVelocityProvider) のスライスを同梱して quad_q へ送信
    ※ logger.info() のフォーマットは従来どおり維持
    dequeにしないとバグる
    """
    # ---------- one-time init ----------
    global ideal_provider
    if 'ideal_provider' not in globals() or ideal_provider is None:
        ideal_provider = IdealVelocityProvider(RUN_SEC, PROC_INTERVAL)

    # ---------- logging ----------
    buf = np.empty((LOG_CHUNK, 3), dtype=np.float32)  # 内部バッファ
    buf_len = 0
    print("buf shape =", buf.shape)

    # --- Python deque リングバッファ ---
    ring_t: deque[np.float32] = deque()
    ring_a: deque[np.float32] = deque()
    ring_b: deque[np.float32] = deque()
    last_A = last_B = None

    next_proc = time.perf_counter()
    cum_count = 0
    last_ts   = next_proc

    while not stop_writer.is_set():
        # ---------- 非ブロッキングで出来るだけ取り込む ----------
        try:
            while True:
                t, pA, pB = buf_q.get_nowait()
                ring_t.extend(t); ring_a.extend(pA); ring_b.extend(pB)
                buf_q.task_done()
        except queue.Empty:
            pass

        # ---------- 次の処理時刻まで待機 ----------
        now = time.perf_counter()
        if now < next_proc:
            time.sleep(next_proc - now)
            continue
        next_proc += PROC_INTERVAL

        # ---------- サンプル不足ならスキップ ----------
        if len(ring_t) < SAMPLES_PROC:
            continue

        # ---------- deque → NumPy へコピー ----------
        t_blk = np.array([ring_t.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        a_blk = np.array([ring_a.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        b_blk = np.array([ring_b.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)

        # ---------- 信号処理 ----------
        dir_log, last_A, last_B = gen_pulse_direction(
            a_blk, b_blk,
            threshold=THRESHOLD_DEFAULT,
            prev_A=last_A, prev_B=last_B
        )
        quad_sig  = gen_quad_pulse(t_blk, dir_log, QUADPULSE_WIDTH, PULSE_HEIGHT, SAMPLE_RATE)
        delta_cnt = pulse_count(dir_log)
        cum_count += delta_cnt
        velocity  = delta_cnt / PROC_INTERVAL / 2048

        # ---------- 理想速度スライス ----------
        t_ref, v_ref = ideal_provider.slice(t_blk[0], t_blk[-1])

        # ---------- ログ ----------
        if DEBUG:
            now = time.perf_counter()
            jitter = (now - last_ts) * 1e3
            logger.info(
                "EPOCH = %f, wall = %6.2f ms, jitter = %6.2f ms  delta c=%+d, v=%6.3f, len(dir_log)=%d, buf_len=%d",
                now, jitter, (now - last_ts) * 1e3,
                delta_cnt, velocity, len(dir_log), buf_len
            )
            last_ts = now

        # ---------- GUI へ送信 ----------
        try:
            quad_q.put_nowait(
                (t_blk, a_blk, b_blk, quad_sig,
                 t_blk[-1], cum_count, velocity,
                 t_ref, v_ref)
            )
        except queue.Full:
            pass

         # ---- append to buffer ----
        buf[buf_len] = (t_blk[-1], (v_ref[-1] if v_ref.size else 0.0), velocity)
        buf_len += 1
        if buf_len == buf.shape[0]:
            n = dset.shape[0]
            dset.resize(n+buf_len, axis=0); dset[-buf_len:] = buf
            buf_len = 0

     # ---- final flush ----
    if buf_len:
        n = dset.shape[0]
        dset.resize(n+buf_len, axis=0); dset[-buf_len:] = buf[:buf_len]
    h5f.close()


def start_gui() -> None:
    pg.setConfigOptions(useOpenGL=True, background="w", foreground="k")
    app = pg.mkQApp("Live plots")

    win = pg.GraphicsLayoutWidget(title="DEMO")

    layout = win.ci.layout                 # GraphicsLayout の中身
    layout.setColumnStretchFactor(0, 3)
    layout.setColumnStretchFactor(1, 5)
    #win_sig = pg.GraphicsLayoutWidget(show=True, title="Signal")
    #win_vel = pg.GraphicsLayoutWidget(show=True, title="Velocity")

    #win_sig.resize(800, 600)
    #win_vel.resize(800, 600)
    #win_sig.show()


    win.resize(800, 600)
    win.show()

    # [0, 0] A/B -------------------------------------------------------
    plt_ab = win.addPlot(row=0, col=0, title="RAW A / B")
    curve_A = plt_ab.plot(pen=pg.mkPen("#ff4b00", width=3))
    curve_B = plt_ab.plot(pen=pg.mkPen("#005aff", width=3))
    plt_ab.setLabel("left", "Amplitude [V]")
    plt_ab.setLabel("bottom", "Time [s]")
    plt_ab.setYRange(-0.5, PULSE_HEIGHT + 0.5)

    # [1,0] Quad waveform --------------------------------------------
    plt_q = win.addPlot(row=1, col=0, title="Quad pulse")
    curve_Q = plt_q.plot(pen=pg.mkPen("m", width=3))
    plt_q.setLabel("left", "Amplitude [V]")
    plt_q.setLabel("bottom", "Time [s]")
    plt_q.setYRange(-PULSE_HEIGHT - 0.5, PULSE_HEIGHT + 0.5)

    # [0,1] count (fixed x-axis) -------------------------------------
    plt_cnt = win.addPlot(row=0, col=1, title="Velovity - Command")
    curve_cnt = plt_cnt.plot(pen=pg.mkPen("#03af7a", width=3))
    #plt_cnt.setXRange(0, RUN_SEC, padding=0)
    #plt_cnt.enableAutoRange("x", True)
    plt_cnt.setLabel("left", "Diff")
    plt_cnt.setLabel("bottom", "Time [s]")

    # [1,1] velocity (fixed x-axis) ----------------------------------
    plt_vel = win.addPlot(row=1, col=1, title="Velocity")
    curve_vel     = plt_vel.plot(pen=pg.mkPen("#00a0e9", width=3))   # measured
    curve_vel_ref = plt_vel.plot(pen=pg.mkPen("#a05aff", width=3))   # ideal (new)
    #plt_vel.setXRange(0, RUN_SEC, padding=0)
    #plt_vel.enableAutoRange("x", True)
    plt_vel.setLabel("left", "Velocity [rps]")
    plt_vel.setLabel("bottom", "Time [s]")

    # buffers ---------------------------------------------------------
    xs = ya = yb = yq = np.empty(0, dtype=np.float32)
    xs_cnt = y_cnt = np.empty(0, dtype=np.float32)
    xs_vel = y_vel = np.empty(0, dtype=np.float32)
    xr = yr = np.empty(0, dtype=np.float32)       # ideal velocity buffers (new)

    def refresh():
        nonlocal xs, ya, yb, yq, xs_cnt, y_cnt, xs_vel, y_vel, xr, yr
        try:
            while True:
                # receive processed data (+ ideal velocity slice)
                t_ax, pA, pB, qsig, t_end, cum_cnt, vel, t_ref, v_ref = quad_q.get_nowait()

                xs = np.concatenate((xs, t_ax))[-HISTORY:]
                ya = np.concatenate((ya, pA))[-HISTORY:]
                yb = np.concatenate((yb, pB))[-HISTORY:]
                yq = np.concatenate((yq, qsig))[-HISTORY:]

                xs_cnt = np.append(xs_cnt, t_end)[-COUNT_HISTORY:]
                y_cnt  = np.append(y_cnt, vel-v_ref)[-COUNT_HISTORY:]

                xs_vel = np.append(xs_vel, t_end)[-VELO_HISTORY:]
                y_vel  = np.append(y_vel, vel)[-VELO_HISTORY:]

                xr = np.concatenate((xr, t_ref))[-VELO_HISTORY:]
                yr = np.concatenate((yr, v_ref))[-VELO_HISTORY:]

                quad_q.task_done()
        except queue.Empty:
            pass

        # scrolling window for waveforms only
        if xs.size:
            start = xs[-1] - PLOT_SEC
            plt_ab.setXRange(start, xs[-1], padding=0)
            plt_q.setXRange(start, xs[-1], padding=0)

        # --- auto-range for count/velocity ---
        if xs_cnt.size:
            plt_cnt.setXRange(xs_cnt[-1]-10, xs_cnt[-1], padding=0)

        if xs_vel.size:
            plt_vel.setXRange(xs_vel[-1]-10, xs_vel[-1], padding=0)

        # --- push data to curves ---
        curve_A.setData(xs, ya)
        curve_B.setData(xs, yb)
        curve_Q.setData(xs, yq)
        curve_cnt.setData(xs_cnt, y_cnt)
        curve_vel.setData(xs_vel, y_vel)
        curve_vel_ref.setData(xr, yr)

    timer = QtCore.QTimer()
    timer.timeout.connect(refresh)
    timer.start(GUI_INTERVAL_MS)

    # auto-stop after RUN_SEC
    QtCore.QTimer.singleShot(int(RUN_SEC * 1000), lambda: (stop_writer.set(), app.quit()))
    app.exec()


In [5]:
if __name__ == "__main__":
    threading.Thread(target=log_listener, daemon=True).start()  # start log listener

    gen_th  = threading.Thread(target=generator, daemon=True)
    proc_th = threading.Thread(target=processor, daemon=True)

    gen_th.start()
    proc_th.start()
    # con_th.start()

    start_gui()  # blocks until the user closes the window or timer expires

    # join threads and exit
    stop_writer.set()
    gen_th.join()
    proc_th.join()

    print("Graceful shutdown.")


buf shape = (1024, 3)
EPOCH = 431566.609173, wall = 955.38 ms, jitter = 955.38 ms  delta c=+25, v= 0.098, len(dir_log)=12500, buf_len=0
EPOCH = 431566.702873, wall =  93.70 ms, jitter =  93.70 ms  delta c=+26, v= 0.102, len(dir_log)=12500, buf_len=1
EPOCH = 431566.723712, wall =  20.84 ms, jitter =  20.84 ms  delta c=+25, v= 0.098, len(dir_log)=12500, buf_len=2
EPOCH = 431566.736137, wall =  12.42 ms, jitter =  12.42 ms  delta c=+26, v= 0.102, len(dir_log)=12500, buf_len=3
EPOCH = 431567.061252, wall = 325.12 ms, jitter = 325.12 ms  delta c=+25, v= 0.098, len(dir_log)=12500, buf_len=4
EPOCH = 431567.185815, wall = 124.56 ms, jitter = 124.56 ms  delta c=+26, v= 0.102, len(dir_log)=12500, buf_len=5
EPOCH = 431567.289527, wall = 103.71 ms, jitter = 103.71 ms  delta c=+26, v= 0.102, len(dir_log)=12500, buf_len=6
EPOCH = 431567.423574, wall = 134.05 ms, jitter = 134.05 ms  delta c=+25, v= 0.098, len(dir_log)=12500, buf_len=7
EPOCH = 431567.555136, wall = 131.56 ms, jitter = 131.56 ms  delta